In [1]:
from openai import OpenAI
import os
import logging
from jinja2 import Environment, select_autoescape
import json
from miniautogen.pipeline.pipeline import PipelineComponent 

class Jinja2TemplateComponent(PipelineComponent):
    def __init__(self):
        """
        Inicializa o componente sem um template ou variáveis.
        """
        self.template_str = None
        self.variables = None
        self.env = Environment(autoescape=select_autoescape())

    
    def set_template_str(self, template_str):
        """
        Configura a string do template para o componente.

        Args:
            template_str (str): String contendo o template Jinja2.
        """
        self.template_str = template_str
    
    def set_variables(self, variables):
        """
        Configura as variáveis para o componente.

        Args:
            variables (dict): Dicionário de variáveis para o template.
        """
        self.variables = variables

    def process(self, state):
        """
        Processa o estado atual do pipeline, substituindo as variáveis no template.

        Args:
            state (PipelineState): Estado atual do pipeline contendo as variáveis.

        Returns:
            PipelineState: Estado atualizado do pipeline.
        """
        template = self.env.from_string(self.template_str)
        chat = state.get_state().get('group_chat')
        agent = state.get_state().get('selected_agent')
        messages = chat.get_messages()[['sender_id', 'message']].to_dict(orient='records')

        # Verifica se as variáveis foram definidas
        if self.variables is None:
            self.variables = state.get_state().get('variables', {})

        self.variables['chat'] = chat
        self.variables['agent'] = agent
        self.variables['messages'] = messages
        # Renderiza o template com as variáveis fornecidas
        prompt = template.render(self.variables)
        prompt = json.loads(prompt)
        print(prompt)

        # Atualiza o estado do pipeline com a saída renderizada
        state.update_state(prompt=prompt)

        return state
    

class OpenAIComponent(PipelineComponent):
    """
    Componente de Pipeline para gerar respostas utilizando o modelo de linguagem da OpenAI.
    """

    def __init__(self):
        self.client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
        self.logger = logging.getLogger(__name__)

    def process(self, state):
        try:
            prompt = state.get_state().get('prompt')
            if not prompt:
                raise ValueError(
                    "groupchat e agent são obrigatórios para OpenAIResponseComponent.")
            response = self._call_openai_api(prompt)
            return response.choices[0].message.content
        except Exception as e:
            self.logger.error(f"Erro em OpenAIResponseComponent: {e}")
            raise

    def _call_openai_api(self, prompt):
        """ Realiza a chamada à API da OpenAI. """
        try:
            return self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=prompt,
                temperature=1
            )
        except Exception as e:
            self.logger.error(f"Erro ao chamar a API da OpenAI: {e}")
            raise


In [2]:
from miniautogen.chat.chat import Chat
from miniautogen.agent.agent import Agent
from miniautogen.chat.chatadmin import ChatAdmin
from miniautogen.pipeline.pipeline import Pipeline
from miniautogen.pipeline.components import (
    UserResponseComponent,
    NextAgentSelectorComponent,
    AgentReplyComponent,
    TerminateChatComponent
)

template_str = """
[
  {"role": "system", "content": "{{ agent.role }}"}{% for message in messages %},
  {% if message.sender_id == agent.agent_id %}
    {"role": "assistant", "content": {{ message.message | tojson | safe }}}
  {% else %}
    {"role": "user", "content": {{ message.message | tojson | safe }}}
  {% endif %}
{% endfor %}
]
"""

# Cria uma instância do componente com o template
jinja_component = Jinja2TemplateComponent()
jinja_component.set_template_str(template_str)

# Configuração dos Pipelines
pipeline_user = Pipeline([UserResponseComponent()])
pipeline_jinja = Pipeline([jinja_component, OpenAIComponent()])
pipeline_admin = Pipeline(
    [NextAgentSelectorComponent(), AgentReplyComponent(), TerminateChatComponent()])

# Setup do ambiente de teste
chat = Chat()
json_data = {
    'agent_id': 'Bruno',
    'name': 'Bruno',
    'role': 'user'
}

# Criação de Agentes
agent1 = Agent.from_json(json_data)
agent1.pipeline = pipeline_user  # Atribuindo o pipeline ao agente

agent2 = Agent("dev", "Carlos", "Python senior Developer")
agent2.pipeline = pipeline_jinja  # Atribuindo o pipeline_llm ao segundo agente


# Adicionando os agentes ao chat
chat.add_agent(agent1)
chat.add_agent(agent2)

# Adição de mensagens de teste ao chat
json_messages = [
    {'sender_id': 'user4', 'message': 'Que bom ouvir isso!'},
    {'sender_id': 'user5', 'message': 'Vamos continuar conversando.',
        'additional_info': {'topic': 'chat'}},
]
chat.add_messages(json_messages)

# Criação e configuração do ChatAdmin
chat_admin = ChatAdmin("admin", "Admin", "admin_role",
                       pipeline_admin, chat, "manage_chat", 10)

In [3]:
chat_admin.run()

INFO:miniautogen.chat.chatadmin:Chat Admin started.
INFO:miniautogen.chat.chatadmin:Executing round 1
INFO:miniautogen.chat.chatadmin:Executing round 2
INFO:miniautogen.chat.chatadmin:Executing round 3


Voce sabe python?
Erro ao processar a resposta do agente: Variáveis não fornecidas para o componente Jinja2Template.
[{'role': 'system', 'content': 'Python senior Developer'}, {'role': 'user', 'content': 'Que bom ouvir isso!'}, {'role': 'user', 'content': 'Vamos continuar conversando.'}, {'role': 'user', 'content': 'Que bom ouvir isso!'}, {'role': 'user', 'content': 'Vamos continuar conversando.'}, {'role': 'user', 'content': 'Olá, quem é voce?'}, {'role': 'assistant', 'content': 'Olá, sou um Assistente de IA programado para ajudar a responder perguntas e fornecer informações. Como posso ajudá-lo hoje?'}, {'role': 'user', 'content': 'Qual seu system?'}, {'role': 'assistant', 'content': 'Eu sou um Assistente de IA baseado em linguagem natural. Fui desenvolvido utilizando técnicas de processamento de linguagem natural e aprendizado de máquina para entender e responder perguntas dos usuários. Estou aqui para ajudar com qualquer dúvida que você possa ter.'}, {'role': 'user', 'content': 'te

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:miniautogen.chat.chatadmin:Executing round 4


Sim, sou um Assistente de IA programado em Python. Tenho conhecimento em várias bibliotecas e frameworks populares de Python, e posso ajudar com dúvidas e problemas relacionados à linguagem. Como posso ajudar você com Python?


INFO:miniautogen.chat.chatadmin:Chat Admin stopped.
INFO:miniautogen.chat.chatadmin:Chat Admin stopped.


terminate
Encerrando chat...


In [4]:
agent2.__dict__

{'agent_id': 'dev',
 'name': 'Carlos',
 'role': 'Python senior Developer',
 'pipeline': <miniautogen.pipeline.pipeline.Pipeline at 0x13832d190>,
 'status': 'ativo'}

In [5]:
chat.__dict__

{'storage_path': 'groupchat_data',
 'db_path': 'groupchat_data/groupchat.db',
 'storage': <miniautogen.storage.chatstorage.ChatStorage at 0x13832d410>,
 'agentList': [<miniautogen.agent.agent.Agent at 0x13832d4d0>,
 'context_path': 'groupchat_data/context.json',
 'context': {},
 'custom_df': None}

In [7]:
chat.get_messages().to_dict(orient='records')

[{'id': 1,
  'sender_id': 'user4',
  'message': 'Que bom ouvir isso!',
  'timestamp': Timestamp('2023-12-29 05:43:02.807385'),
  'additional_info': None},
 {'id': 2,
  'sender_id': 'user5',
  'message': 'Vamos continuar conversando.',
  'timestamp': Timestamp('2023-12-29 05:43:02.809925'),
  'additional_info': {'topic': 'chat'}}]